In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn 
from sklearn import preprocessing 
from boruta import BorutaPy

Fetching the data from the url

In [49]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

Feature Engineering

In [50]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['time']
del df['datenew']
#SETTING DATE COLUMN AS INDEX
df = df.set_index('date')

len 139


Final DataFrame

In [51]:
df.head(10)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,...,18,11,0,66000,0,0,0,0,0,0


Boruta without Scaling

In [52]:
data1 = df.drop('Appliances', 1)
data2 = df.Appliances.values
x=data1.values
y=data2

In [53]:

 

rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features 
feat_selector.fit(x, y)






Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	10
Tentative: 	8
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	10
Tentative: 	8
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	10
Tentative: 	8
Rejected: 	20


...

Iteration: 	97 / 100
Confirmed: 	17
Tentative: 	1
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	98 / 100
Confirmed: 	17
Tentative: 	1
Rejected: 	20
Iteration: 	99 / 100
Confirmed: 	17
Tentative: 	1
Rejected: 	20


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	17
Tentative: 	1
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


BorutaPy(alpha=0.05,
     estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=120, n_jobs=-1,
           oob_score=False,
           random_state=<mtrand.RandomState object at 0x0000017F3468D288>,
           verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x0000017F3468D288>,
     two_step=True, verbose=2)

In [59]:
# check selected features 
print("Mask of n features with rank 1")
print(data1.columns.values)
print(feat_selector.support_)
print("######################")
# check ranking of features
print("Rank of features")
print(data1.columns.values)
print(feat_selector.ranking_)
print("#######################")
print("Number of features selected")
print (feat_selector.n_features_)

Mask of n features with rank 1
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[ True False False False  True  True  True False  True  True  True False
  True False  True  True False  True False False  True False  True False
  True False False  True  True False False  True False False False False
 False False]
######################
Rank of features
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[ 1  9 10  6  1  1  1  3  1  1  1  5  1  3  1  1  7  1  8  2  1  3  1 12
  1 14 

In [61]:
featuredf = pd.DataFrame(data1.columns.tolist(), columns=['features'])
print("################")
print(featuredf)
featuredf['rank']=feat_selector.ranking_
featuredf = featuredf.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))
print("Ranking of all the features")
print (feature_df)

################
         features
0          lights
1              T1
2            RH_1
3              T2
4            RH_2
5              T3
6            RH_3
7              T4
8            RH_4
9              T5
10           RH_5
11             T6
12           RH_6
13             T7
14           RH_7
15             T8
16           RH_8
17             T9
18           RH_9
19          T_out
20    Press_mm_hg
21         RH_out
22      Windspeed
23     Visibility
24      Tdewpoint
25            rv1
26            rv2
27          month
28           hour
29            day
30  weekendstatus
31            NSM
32         Friday
33       Saturday
34         Sunday
35       Thursday
36        Tuesday
37      Wednesday

 Top 17 features:
       features  rank
0        lights     1
1   Press_mm_hg     1
2         month     1
3            T9     1
4            T8     1
5          RH_7     1
6          RH_6     1
7     Windspeed     1
8          RH_5     1
9     Tdewpoint     1
10         RH_4     

In [62]:
#check weak features
print ('\n Support for weak features:')
print (data1.columns.values)
print (feat_selector.support_weak_)


 Support for weak features:
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False]


Boruta on scaled data

In [65]:
#Using MinMax Scalar
y1 = y.reshape((len(y), 1))
x1 = x

In [70]:
min_max_scaler = preprocessing.MinMaxScaler()
X1 = min_max_scaler.fit_transform(x1)
Y1 = min_max_scaler.fit_transform(y1)


C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [71]:
df['yscaled'] = Y1
Y1 = df.yscaled.values

In [75]:
rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', max_iter=120, verbose=2, random_state=1)

# find all relevant features 
feat_selector.fit(X1, Y1)



Iteration: 	1 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	2 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	3 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	4 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	5 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	6 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	7 / 120
Confirmed: 	0
Tentative: 	38
Rejected: 	0
Iteration: 	8 / 120
Confirmed: 	10
Tentative: 	8
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 120
Confirmed: 	10
Tentative: 	8
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 120
Confirmed: 	10
Tentative: 	8
Rejected: 	20


...

Iteration: 	117 / 120
Confirmed: 	17
Tentative: 	1
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	118 / 120
Confirmed: 	17
Tentative: 	1
Rejected: 	20
Iteration: 	119 / 120
Confirmed: 	17
Tentative: 	1
Rejected: 	20


BorutaPy finished running.

Iteration: 	120 / 120
Confirmed: 	17
Tentative: 	1
Rejected: 	20


C:\Users\chethan\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


BorutaPy(alpha=0.05,
     estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=120, n_jobs=-1,
           oob_score=False,
           random_state=<mtrand.RandomState object at 0x0000017F3D3ADF30>,
           verbose=0, warm_start=False),
     max_iter=120, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x0000017F3D3ADF30>,
     two_step=True, verbose=2)

In [76]:
# check selected features 
print("Mask of n features with rank 1")
print(data1.columns.values)
print(feat_selector.support_)
print("######################")
# check ranking of features
print("Rank of features")
print(data1.columns.values)
print(feat_selector.ranking_)
print("#######################")
print("Number of features selected")
print (feat_selector.n_features_)

Mask of n features with rank 1
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[ True False False False  True  True  True False  True  True  True False
  True False  True  True False  True False False  True False  True False
  True False False  True  True False False  True False False False False
 False False]
######################
Rank of features
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[ 1  9 10  6  1  1  1  3  1  1  1  5  1  3  1  1  7  1  8  2  1  3  1 12
  1 14 

In [77]:
featuredf = pd.DataFrame(data1.columns.tolist(), columns=['features'])
print("################")
print(featuredf)
featuredf['rank']=feat_selector.ranking_
featuredf = featuredf.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))
print("Ranking of all the features")
print (feature_df)

################
         features
0          lights
1              T1
2            RH_1
3              T2
4            RH_2
5              T3
6            RH_3
7              T4
8            RH_4
9              T5
10           RH_5
11             T6
12           RH_6
13             T7
14           RH_7
15             T8
16           RH_8
17             T9
18           RH_9
19          T_out
20    Press_mm_hg
21         RH_out
22      Windspeed
23     Visibility
24      Tdewpoint
25            rv1
26            rv2
27          month
28           hour
29            day
30  weekendstatus
31            NSM
32         Friday
33       Saturday
34         Sunday
35       Thursday
36        Tuesday
37      Wednesday

 Top 17 features:
       features  rank
0        lights     1
1   Press_mm_hg     1
2         month     1
3            T9     1
4            T8     1
5          RH_7     1
6          RH_6     1
7     Windspeed     1
8          RH_5     1
9     Tdewpoint     1
10         RH_4     

In [78]:
#check weak features
print ('\n Support for weak features:')
print (data1.columns.values)
print (feat_selector.support_weak_)


 Support for weak features:
['lights' 'T1' 'RH_1' 'T2' 'RH_2' 'T3' 'RH_3' 'T4' 'RH_4' 'T5' 'RH_5' 'T6'
 'RH_6' 'T7' 'RH_7' 'T8' 'RH_8' 'T9' 'RH_9' 'T_out' 'Press_mm_hg' 'RH_out'
 'Windspeed' 'Visibility' 'Tdewpoint' 'rv1' 'rv2' 'month' 'hour' 'day'
 'weekendstatus' 'NSM' 'Friday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday'
 'Wednesday']
[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False]
